In [1]:
!pip install streamlit streamlit_option_menu

In [2]:
API_Key = "NZuKNoxEadqDrXu1uaGhrO0FD0b8RTErPk4r42Ra"

import requests

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_Key}"

response = requests.get(url)

data = response.json()

details = data["near_earth_objects"]

asteroids_data = []
target = 10000

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_Key}"

while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data["near_earth_objects"]
    for date, asteroids in details.items():
        for asteroid in asteroids:
            asteroids_data.append(dict(id= asteroid['id'],
                                   neo_reference_id= asteroid['neo_reference_id'],
                                   name= asteroid['name'],
                                   absolute_magnitude_h= asteroid['absolute_magnitude_h'],
                                   estimated_diameter_min_km= asteroid['estimated_diameter']['kilometers']['estimated_diameter_min'],
                                   estimated_diameter_max_km= asteroid['estimated_diameter']['kilometers']['estimated_diameter_max'],
                                   is_potentially_hazardous_asteroid= asteroid['is_potentially_hazardous_asteroid'],
                                   close_approach_date= asteroid['close_approach_data'][0]['close_approach_date'],
                                   relative_velocity_kmph= float(asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                                   astronomical= float(asteroid['close_approach_data'][0]['miss_distance']['astronomical']),
                                   miss_distance_km= float(asteroid['close_approach_data'][0]['miss_distance']['kilometers']),
                                   miss_distance_lunar= float(asteroid['close_approach_data'][0]['miss_distance']['lunar']),
                                   orbiting_body= asteroid['close_approach_data'][0]['orbiting_body']))
            if len(asteroids_data) >= target:
                break  # Break the inner loop once the target is reached
        if len(asteroids_data) >= target:
            break # Break the outer loop once the target is reached
    if len(asteroids_data) >= target:
            break # Break the while loop once the target is reached
    url = data['links']['next']

from datetime import datetime
for obj in asteroids_data:
    # Convert date string to datetime.date
    obj['close_approach_date'] = datetime.strptime(obj['close_approach_date'], "%Y-%m-%d").date()

import sqlite3

# Connect to a database file (creates it if it doesn't exist)
conn = sqlite3.connect('NASA_NEO_track')

cursor = conn.cursor()

import pandas as pd
cursor.execute('''CREATE TABLE Asteroids(
    id INT,
    name TEXT,
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)''')
cursor.execute('''CREATE TABLE Close_approach(
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR
)''')
for obj in asteroids_data:
  cursor.execute('''INSERT INTO Asteroids(id,
                                          name,
                                          absolute_magnitude_h,
                                          estimated_diameter_min_km,
                                          estimated_diameter_max_km,
                                          is_potentially_hazardous_asteroid)
  VALUES(?, ?, ?, ?, ?, ?)''',(obj['id'],
                               obj['name'],
                               obj['absolute_magnitude_h'],
                               obj['estimated_diameter_min_km'],
                               obj['estimated_diameter_max_km'],
                               obj['is_potentially_hazardous_asteroid']))

  cursor.execute('''INSERT INTO Close_approach(neo_reference_id,
                                              close_approach_date,
                                              relative_velocity_kmph,
                                              astronomical,
                                              miss_distance_km,
                                              miss_distance_lunar,
                                              orbiting_body)
  VALUES(?, ?, ?, ?, ?, ?, ?)''',(obj['neo_reference_id'],
                                  obj['close_approach_date'],
                                  obj['relative_velocity_kmph'],
                                  obj['astronomical'],
                                  obj['miss_distance_km'],
                                  obj['miss_distance_lunar'],
                                  obj['orbiting_body']))

conn.commit()

In [45]:
%%writefile neo_dashboard.py

Overwriting neo_dashboard3.py


In [57]:
%%writefile neo_dashboard.py

import streamlit as st
import pandas as pd
import sqlite3


st.set_page_config(layout="wide", page_title="NASA NEO Explorer")
st.title("🌌 NASA Near Earth Object Dashboard")


conn = sqlite3.connect("NASA_NEO_track")


st.sidebar.header("📚 Predefined SQL Queries")

query_dict = {
    "1. Count how many times each asteroid has approached Earth": "SELECT neo_reference_id, COUNT(*) AS Approach_count FROM close_approach GROUP BY neo_reference_id ORDER BY Approach_count DESC",
    "2. Average velocity of each asteroid over multiple approaches": "SELECT ca.neo_reference_id, a.name, ROUND(AVG(ca.relative_velocity_kmph), 2) AS avg_velocity_kmph FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id GROUP BY ca.neo_reference_id ORDER BY avg_velocity_kmph DESC",
    "3. Top 10 Fastest Asteroids": "SELECT ca.neo_reference_id, a.name, MAX(ca.relative_velocity_kmph) AS max_velocity_kmph FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id GROUP BY ca.neo_reference_id ORDER BY max_velocity_kmph DESC LIMIT 10",
    "4. Potentially hazardous asteroids that have approached Earth more than 3 times": "SELECT a.name, COUNT(*) AS approach_count FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id WHERE a.is_potentially_hazardous_asteroid = 1 GROUP BY ca.neo_reference_id HAVING approach_count > 3",
    "5. Month with the most asteroid approaches": "SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS total_approaches FROM close_approach GROUP BY month ORDER BY total_approaches DESC LIMIT 1",
    "6. Asteroid with the fastest ever approach speed": "SELECT a.name, ca.relative_velocity_kmph, ca.close_approach_date FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id ORDER BY ca.relative_velocity_kmph DESC LIMIT 1",
    "7. Sort asteroids by maximum estimated diameter": "SELECT name, estimated_diameter_max_km FROM asteroids ORDER BY estimated_diameter_max_km DESC LIMIT 10",
    "8. An asteroid whose closest approach is getting nearer over time": "SELECT a.name, c.close_approach_date, MIN(c.miss_distance_km) AS min_distance FROM close_approach c JOIN asteroids a ON a.id = c.neo_reference_id GROUP BY a.id ORDER BY min_distance ASC",
    "9. Display the name of each asteroid along with the date and miss distance of its closest approach to Earth": "SELECT a.name, c.close_approach_date, c.miss_distance_km FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id ORDER BY c.miss_distance_km ASC",
    "10. List names of asteroids that approached Earth with velocity > 50,000 km/h": "SELECT a.name, c.relative_velocity_kmph FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id WHERE c.relative_velocity_kmph > 50000 ORDER BY c.relative_velocity_kmph DESC",
    "11. Count how many approaches happened per month": "SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count FROM close_approach GROUP BY month ORDER BY month",
    "12. Find asteroid with the highest brightness (lowest magnitude value)": "SELECT name, absolute_magnitude_h FROM asteroids ORDER BY absolute_magnitude_h ASC LIMIT 1",
    "13. Get number of hazardous vs non-hazardous asteroids": "SELECT is_potentially_hazardous_asteroid AS hazardous, COUNT(*) FROM asteroids GROUP BY is_potentially_hazardous_asteroid",
    "14. Approaches <1 Lunar Distance": "SELECT a.name, c.close_approach_date, c.miss_distance_lunar FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id WHERE c.miss_distance_lunar < 1 ORDER BY c.miss_distance_lunar ASC",
    "15. Approaches <0.05 AU": "SELECT a.name, c.close_approach_date, c.astronomical FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id WHERE c.astronomical < 0.05 ORDER BY c.astronomical ASC",
    "16. Asteroids with Multiple Approaches": "SELECT a.name, COUNT(*) AS approach_count FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id GROUP BY a.id HAVING approach_count > 1 ORDER BY approach_count DESC",
    "16. Top 10 Largest Asteroids (by estimated max diameter)": "SELECT name, estimated_diameter_max_km FROM asteroids ORDER BY estimated_diameter_max_km DESC LIMIT 10",
    "17. Closest approach per asteroid (their nearest pass)": "SELECT a.name, MIN(c.close_approach_date) AS closest_approach FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id GROUP BY a.id ORDER BY closest_approach ASC",
    "18.  Monthly trend of potentially hazardous approaches": "SELECT strftime('%Y-%m', c.close_approach_date) AS month, COUNT(*) AS hazardous_approaches FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id WHERE a.is_potentially_hazardous_asteroid = 1 GROUP BY month ORDER BY month",
    "19. Asteroids that have multiple close approaches": "SELECT a.name, COUNT(*) AS approach_count FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id GROUP BY a.id HAVING approach_count > 1 ORDER BY approach_count DESC",
    "20. Fastest 5 Asteroids during approach": "SELECT a.name, c.relative_velocity_kmph FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id ORDER BY c.relative_velocity_kmph DESC LIMIT 5"
}

selected_query = st.sidebar.selectbox("Choose a Query", list(query_dict.keys()))
if selected_query:
    st.subheader(f"📊 Result: {selected_query}")
    df_query = pd.read_sql_query(query_dict[selected_query], conn)
    st.dataframe(df_query, use_container_width=True)


st.header("🎛️ Custom Filtered View")


min_date = pd.read_sql_query("SELECT MIN(close_approach_date) FROM close_approach", conn).iloc[0, 0]
max_date = pd.read_sql_query("SELECT MAX(close_approach_date) FROM close_approach", conn).iloc[0, 0]

col1, col2 = st.columns(2)
with col1:
    date_range = st.date_input("📅 Close Approach Date", [pd.to_datetime(min_date), pd.to_datetime(max_date)])
    au_range = st.slider("🌍 Astronomical Distance (AU)", 0.0, 1.0, (0.0, 0.5))
    ld_range = st.slider("🌕 Lunar Distance", 0.0, 100.0, (0.0, 10.0))
with col2:
    velocity_range = st.slider("🚀 Relative Velocity (km/h)", 0, 150000, (0, 80000))
    diameter_range = st.slider("📏 Estimated Diameter (km)", 0.0, 5.0, (0.0, 2.0))
    hazardous_state = st.selectbox("☠️ Hazardous?", ("All", "Hazardous Only", "Non-Hazardous"))

# Construct dynamic SQL
query = f"""
    SELECT a.name, c.close_approach_date, c.relative_velocity_kmph,
           c.miss_distance_km, c.miss_distance_lunar, c.astronomical,
           a.estimated_diameter_min_km, a.estimated_diameter_max_km,
           a.is_potentially_hazardous_asteroid
    FROM asteroids a
    JOIN close_approach c ON a.id = c.neo_reference_id
    WHERE c.close_approach_date BETWEEN '{date_range[0]}' AND '{date_range[1]}'
      AND c.astronomical BETWEEN {au_range[0]} AND {au_range[1]}
      AND c.miss_distance_lunar BETWEEN {ld_range[0]} AND {ld_range[1]}
      AND c.relative_velocity_kmph BETWEEN {velocity_range[0]} AND {velocity_range[1]}
      AND a.estimated_diameter_max_km BETWEEN {diameter_range[0]} AND {diameter_range[1]}
"""

if hazardous_state == "Hazardous Only":
    query += " AND a.is_potentially_hazardous_asteroid = 1"
elif hazardous_state == "Non-Hazardous":
    query += " AND a.is_potentially_hazardous_asteroid = 0"

query += " ORDER BY c.relative_velocity_kmph DESC LIMIT 500"

st.markdown("### 📋 Filtered Results")
filtered_df = pd.read_sql_query(query, conn)
st.dataframe(filtered_df, use_container_width=True)


Overwriting neo_dashboard.py


In [40]:
!wget -q -O - ipv4.icanhazip.com

34.75.36.78


In [ ]:
!streamlit run neo_dashboard.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.36.78:8501

⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://big-pumas-rescue.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:26907 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙